In [46]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("output_modified.csv")

In [16]:
selected_data = pd.DataFrame()
selected_data['name'] = data['name']
selected_data['tags'] = data['tags']
selected_data['datasets'] = data['datasets']

In [41]:
selected_data["datasets"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
5902    NaN
5903    NaN
5904    NaN
5905    NaN
5906    NaN
Name: datasets, Length: 5907, dtype: object

In [71]:
def insert_to_map(datamap, dataset, model):
    try:
        values = datamap[dataset]
    except KeyError:
        values = datamap[dataset] = []

    values.append(model)

In [72]:
datasets = []
datasets_map = {}

for index, row in selected_data.iterrows():
    tag_string = str(row["tags"])
    dataset_string = str(row["datasets"])

    if tag_string != "nan":
        tag_list = eval(tag_string)
        for tag in tag_list:
            if "dataset:" in tag:
                tag = tag.removeprefix("dataset:")
                datasets.append(tag)
                insert_to_map(datasets_map, tag, row['name'])
                
    
    if dataset_string != "nan" and dataset_string != "unknown":
        if dataset_string.startswith("["):
            dataset_list = eval(dataset_string)
        else:
            dataset_list = [dataset_string]
        if isinstance(dataset_list, (list, tuple, np.ndarray)):
            for tag in dataset_list:
                # print(tag)
                datasets.append(tag)
                insert_to_map(datasets_map, tag, row['name'])

In [73]:
print(f"Len of datasets before dedup: {len(datasets)}")
datasets = set(datasets)
print(f"Len of datasets after dedup: {len(datasets)}")

Len of datasets before dedup: 7817
Len of datasets after dedup: 791


In [76]:
known_trained_models = set(model for models in datasets_map.values() for model in models)
df = pd.DataFrame(columns=datasets_map.keys(), index=known_trained_models).fillna(0)


for dataset, models in datasets_map.items():
    for model in models:
        df.at[model, dataset] = 1

# Export the DataFrame to CSV
df.to_csv('model_dataset_matrix.csv')

print(df)

                                                    fblgit/simple-math  \
RWKV/rwkv-raven-1b5                                                  0   
cognitivecomputations/TinyDolphin-2.8.2-1.1b-laser                   0   
mwitiderrick/SwahiliInstruct-v0.1                                    0   
nbeerbower/bophades-mistral-math-DPO-7B                              0   
LordNoah/Alpaca_refine_tuned_gpt2_large                              0   
...                                                                ...   
nisten/shqiponja-15b-v1                                              0   
chargoddard/MixtralRPChat-ZLoss                                      0   
heegyu/LIMA2-13b-hf                                                  0   
FuseAI/OpenChat-3.5-7B-Mixtral                                       0   
jondurbin/airoboros-33b-gpt4-1.4                                     0   

                                                    jondurbin/bagel-v0.3  \
RWKV/rwkv-raven-1b5                